In [11]:
import cirq
import numpy as np

# the qubits
qubits = [cirq.GridQubit(0, i) for i in range(3)]

# the state
state = [1, 0, 0]  # Example state: |100>

# the Pauli strings
pauli_strings = ["X0Y1Z2", "Y0Y1Y2", "Z0Z1X2"]

# the expectation values
for pauli_string in pauli_strings:
  # Define the quantum circuit
  circuit = cirq.Circuit()

  # Transform the first qubit to Z
  circuit.append(cirq.H(qubits[0]))

  for i, s in enumerate(state):
    if s == 1:
      # Only apply X gate to initialize |1> state if not the first qubit
      if i != 0:
        circuit.append(cirq.X(qubits[i]))

  # Introduce the upside-down CNOT gate between qubits 0 and 1
  circuit.append(cirq.H(qubits[0]))  # Hadamard on control qubit
  circuit.append(cirq.CNOT(qubits[0], qubits[1]))  # CNOT with control first
  circuit.append(cirq.H(qubits[1]))  # Hadamard on target qubit

  for i in range(0, len(pauli_string), 2):
    gate = pauli_string[i]
    qubit_index = int(pauli_string[i+1])
    if qubit_index == 0:
      # No need to transform the first qubit further
      pass
    else:
      # Apply Identity for other qubits
      circuit.append(cirq.I(qubits[qubit_index]))

  # Add the measurement to the circuit
  circuit.append(cirq.measure(*qubits, key='result'))

  # Simulate the circuit
  simulator = cirq.Simulator()
  result = simulator.run(circuit, repetitions=1000)

  # the expectation value
  expectation_value = np.mean([(-1)**sum(b) for b in result.measurements['result']])

  print(f"The expectation value of the {pauli_string} gate is {expectation_value}")

# Transform the Pauli strings to the Z-basis (this part remains unchanged)
z_basis_pauli_strings = [pauli_string.replace('X0', 'Z0').replace('Y0', 'Z0').replace('Y1', 'I1').replace('Y2', 'I2').replace('Z2','I2').replace('Z1','I1').replace('X2','I2')
                          for pauli_string in pauli_strings]

print("\nThe Pauli strings transformed to the Z-basis are:")
for original, transformed in zip(pauli_strings, z_basis_pauli_strings):
  print(f"{original} -> {transformed}")


The expectation value of the X0Y1Z2 gate is -0.002
The expectation value of the Y0Y1Y2 gate is 0.036
The expectation value of the Z0Z1X2 gate is -0.028

The Pauli strings transformed to the Z-basis are:
X0Y1Z2 -> Z0I1I2
Y0Y1Y2 -> Z0I1I2
Z0Z1X2 -> Z0I1I2
